In [1]:
import os
import fastf1 as ff1
import fastf1.plotting
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error


warnings.filterwarnings('ignore')

# Create cache directory if it doesn't exist
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

ff1.Cache.enable_cache(cache_dir)

In [2]:
schedule = ff1.get_event_schedule(2025)
schedule[schedule['Country'] == 'Australia']

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
1,1,Australia,Melbourne,FORMULA 1 LOUIS VUITTON AUSTRALIAN GRAND PRIX ...,2025-03-16,Australian Grand Prix,conventional,Practice 1,2025-03-14 12:30:00+11:00,2025-03-14 01:30:00,...,Practice 3,2025-03-15 12:30:00+11:00,2025-03-15 01:30:00,Qualifying,2025-03-15 16:00:00+11:00,2025-03-15 05:00:00,Race,2025-03-16 15:00:00+11:00,2025-03-16 04:00:00,True


In [13]:
session = ff1.get_session(2025, 'Las Vegas Grand Prix', 'R')
session.load()

laps = session.laps
laps

core           INFO 	Loading data for Las Vegas Grand Prix - Race [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Fixed incorrect tyre stint information for driver '63'
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '63', '12', '16', '55', '6', '27', '44', '31', '87', '14', '22', '10', '30', '43', '23', '5', '

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 00:59:23.959000,VER,1,0 days 00:01:42.062000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:32.691000,...,True,Red Bull Racing,0 days 00:57:41.627000,2025-11-23 04:04:08.478,12,1.0,False,,False,False
1,0 days 01:01:11.581000,VER,1,0 days 00:01:47.622000,2.0,1.0,NaT,NaT,0 days 00:00:28.482000,0 days 00:00:32.633000,...,True,Red Bull Racing,0 days 00:59:23.959000,2025-11-23 04:05:50.810,216,1.0,False,,False,False
2,0 days 01:03:25.115000,VER,1,0 days 00:02:13.534000,3.0,1.0,NaT,NaT,0 days 00:00:37.904000,0 days 00:00:44.898000,...,True,Red Bull Racing,0 days 01:01:11.581000,2025-11-23 04:07:38.432,6,1.0,False,,False,False
3,0 days 01:05:06.786000,VER,1,0 days 00:01:41.671000,4.0,1.0,NaT,NaT,0 days 00:00:31.896000,0 days 00:00:32.621000,...,True,Red Bull Racing,0 days 01:03:25.115000,2025-11-23 04:09:51.966,671,1.0,False,,False,False
4,0 days 01:06:44.207000,VER,1,0 days 00:01:37.421000,5.0,1.0,NaT,NaT,0 days 00:00:28.002000,0 days 00:00:32.386000,...,True,Red Bull Racing,0 days 01:05:06.786000,2025-11-23 04:11:33.637,1,1.0,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,0 days 02:13:00.598000,PIA,81,0 days 00:01:34.588000,46.0,2.0,NaT,NaT,0 days 00:00:26.951000,0 days 00:00:31.626000,...,True,McLaren,0 days 02:11:26.010000,2025-11-23 05:17:52.861,1,5.0,False,,False,True
882,0 days 02:14:34.684000,PIA,81,0 days 00:01:34.086000,47.0,2.0,NaT,NaT,0 days 00:00:26.763000,0 days 00:00:31.477000,...,True,McLaren,0 days 02:13:00.598000,2025-11-23 05:19:27.449,1,5.0,False,,False,True
883,0 days 02:16:08.849000,PIA,81,0 days 00:01:34.165000,48.0,2.0,NaT,NaT,0 days 00:00:26.769000,0 days 00:00:31.565000,...,True,McLaren,0 days 02:14:34.684000,2025-11-23 05:21:01.535,1,5.0,False,,False,True
884,0 days 02:17:43.315000,PIA,81,0 days 00:01:34.466000,49.0,2.0,NaT,NaT,0 days 00:00:26.922000,0 days 00:00:31.670000,...,True,McLaren,0 days 02:16:08.849000,2025-11-23 05:22:35.700,1,5.0,False,,False,True


In [11]:
def get_event_data(year, event_name):
    '''
    Get qualifying and race data for current year.

    Arguments:
    year -- int, the year of the event
    event_name -- str, the name of the event

    Returns:
    pd.DataFrame -- DataFrame with qualifying times and race lap times from current year
    '''

    # Get free practice data 
    fps = ['FP1', 'FP2', 'FP3']
    fp_results = {}
    for fp in fps:
        try:
            session_fp = ff1.get_session(year, event_name, fp)
            session_fp.load()

            drivers = session_fp.laps['Driver'].unique()
            for driver in drivers:
                if driver not in fp_results:
                    fp_results[driver] = {
                        'driver': driver,
                        'team': session_fp.laps.loc[session_fp.laps['Driver'] == driver, 'Team'].iloc[0],
                    }
                driver_laps = session_fp.laps.pick_driver(driver)

                if not driver_laps.empty:
                    sector_times = driver_laps[['Sector1Time', 'Sector2Time', 'Sector3Time']].mean().dropna()             
                    total_times = sector_times.sum().total_seconds() if not sector_times.isna().any() else None

                    fp_results[driver] = {
                        **fp_results[driver],
                        f'{fp}_total_time': total_times
                    }
        except Exception as e:
            print(f"Could not load {fp} for {event_name} {year}: {e}")
            continue
    fp_df = pd.DataFrame.from_dict(fp_results, orient='index')

    # Get current year qualifying data
    try:
        session_q = ff1.get_session(year, event_name, 'Q')
        session_q.load()
        
        qualifying_laps = session_q.laps[['Driver', 'LapTime']].copy()
        qualifying_laps = qualifying_laps.dropna(subset=['LapTime'])
        
        # Get best qualifying lap per driver
        qualifying_best = qualifying_laps.groupby('Driver')['LapTime'].min().reset_index()
        qualifying_best['Qualifying_Time'] = qualifying_best['LapTime'].dt.total_seconds()
        qualifying_best = qualifying_best[['Driver', 'Qualifying_Time']]
        
    except Exception as e:
        print(f"Could not load qualifying for {event_name} {year}: {e}")
        return pd.DataFrame()
    
    # Get current year race data
    try:
        session_r = ff1.get_session(year, event_name, 'R')
        session_r.load()
        
        race_laps = session_r.laps.copy()
        race_laps = race_laps.dropna(subset=['LapTime'])
        race_laps['Race_Time'] = race_laps['LapTime'].dt.total_seconds()
        race_laps = race_laps[['Driver', 'Race_Time', 'LapNumber']]

        # Add weather condition
        weather = session_r.weather_data[['AirTemp', 'TrackTemp', 'Humidity', 'WindSpeed', 'Rainfall', 'Pressure']]
        if weather is not None and not weather.empty:
            for col in weather.columns:
                race_laps[f'Weather_{col}'] = weather[col].mean()
        
    except Exception as e:
        print(f"Could not load race for {event_name} {year}: {e}")
        return pd.DataFrame()
    
    # Merge qualifying and race data
    merged_df = qualifying_best.merge(race_laps, on='Driver', how='inner')
    merged_df = merged_df.merge(fp_df, left_on='Driver', right_on='driver', how='left')
    merged_df.drop(columns=['driver'], inplace=True)
    merged_df['Year'] = year
    merged_df['EventName'] = event_name
    
    return merged_df

In [12]:
australian_gp_2025_data = get_event_data(2025, 'Australian Grand Prix')
australian_gp_2025_data

core           INFO 	Loading data for Australian Grand Prix - Practice 1 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '5', '6', '7', '10', '12', '14', '16', '18', '22', '23', '27', '30', '31', '44', '55', '63', '81', '87']
core           INFO 	Loading data for Australian Grand Prix - Practice 2 [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for t

,Driver,Qualifying_Time,Race_Time,LapNumber,Weather_AirTemp,Weather_TrackTemp,Weather_Humidity,Weather_WindSpeed,Weather_Rainfall,Weather_Pressure,team,FP1_total_time,FP2_total_time,FP3_total_time,Year,EventName
0,ALB,75.737,132.195,1.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Williams,107.037677,100.042091,105.053031,2025,Australian Grand Prix
1,ALB,75.737,142.084,5.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Williams,107.037677,100.042091,105.053031,2025,Australian Grand Prix
2,ALB,75.737,139.612,6.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Williams,107.037677,100.042091,105.053031,2025,Australian Grand Prix
3,ALB,75.737,129.065,7.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Williams,107.037677,100.042091,105.053031,2025,Australian Grand Prix
4,ALB,75.737,98.281,8.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Williams,107.037677,100.042091,105.053031,2025,Australian Grand Prix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,VER,75.481,90.859,53.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Red Bull Racing,104.157015,100.398777,109.207585,2025,Australian Grand Prix
801,VER,75.481,89.100,54.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Red Bull Racing,104.157015,100.398777,109.207585,2025,Australian Grand Prix
802,VER,75.481,88.538,55.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Red Bull Racing,104.157015,100.398777,109.207585,2025,Australian Grand Prix
803,VER,75.481,87.921,56.0,15.707865,18.942135,78.421348,3.475281,0.325843,1009.901685,Red Bull Racing,104.157015,100.398777,109.207585,2025,Australian Grand Prix


In [6]:
# Gradient Boosting Regressor model to predict race
X = australian_gp_2025_data[['Qualifying_Time']]
y = australian_gp_2025_data['Race_Time']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 16.139764543693552


In [7]:
results_df = pd.DataFrame({
    'Driver': australian_gp_2025_data.loc[X_test.index, 'Driver'],
    'Actual': y_test,
    'Predicted': y_pred
})

In [8]:
results_df


,Driver,Actual,Predicted
192,GAS,93.124,103.627999
718,TSU,91.793,105.685314
168,BOR,132.146,100.531078
522,OCO,102.035,103.082319
536,OCO,93.152,103.082319
...,...,...,...
659,STR,93.631,105.950216
356,LAW,92.883,102.474583
525,OCO,88.505,103.082319
749,TSU,91.891,105.685314
